# 1. Thư viện

In [1]:
import os
import json

from dotenv import load_dotenv
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain.docstore.document import Document
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import Distance, VectorParams
from langchain_qdrant import QdrantVectorStore, RetrievalMode

d:\Download_Python\lib\importlib\__init__.py:127: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  return _bootstrap._gcd_import(name[level:], package, level)


# 2. Model embedding

## 2.1. Dùng model embedding gpt4all từ langchain

In [2]:
model_name = "nomic-embed-text-v1.f16.gguf"
gpt4all_kwargs = {'allow_download': 'True'}

model_Dense_Embedding = GPT4AllEmbeddings(
    model_name=model_name,
    gpt4all_kwargs=gpt4all_kwargs,
    )

## 2.2. Dùng model Viet-Mistral/Vistral-7B-Chats

In [3]:
# nothing here

# 3. Tạo Collection

In [4]:
load_dotenv()

URL_QDRANT = os.getenv("URL_QDRANT")
API_QDRANT = os.getenv("API_QDRANT")

In [5]:
url = URL_QDRANT
api_key = API_QDRANT

## 3.1. Load file data

In [6]:
# Tải dữ liệu từ file JSON
with open(r'data\trich_dan_luat\json\test_Combined_Output_Keywords.json', 'r', encoding='utf-8') as f:
    data_list = json.load(f)

## 3.2. Tạo các documents từ data

In [7]:
documents = []
for data in data_list:
    # Lấy page_content từ key "combine_Article_Content"
    page_content = data.get("combine_Article_Content", "")
    
    # Tạo metadata từ các key khác, loại trừ "combine_Article_Content"
    metadata = {key: value for key, value in data.items() if key != "combine_Article_Content"}
    
    # Tạo Document và thêm vào danh sách documents
    doc = Document(page_content=page_content, metadata=metadata)
    documents.append(doc)

In [8]:
# In ra 3 document đầu tiên
for doc in documents[:3]:
    print("Page Content:", doc.page_content)
    print("Metadata:", doc.metadata)
    print("-" * 50)  # Để phân cách giữa các document

Page Content: Điều 1. Phạm vi điều chỉnh
1. Luật này quy định việc thực hiện giao dịch bằng phương tiện điện tử.
Metadata: {'stt': '10', 'loai_van_ban': 'Luật', 'noi_ban_hanh': 'Quốc hội', 'so_hieu': '20/2023/QH15', 'linhvuc_nganh': 'Thương mại, Công nghệ thông tin, Quyền dân sự', 'ngay_ban_hanh': '22/06/2023', 'ngay_hieu_luc': '01/07/2024', 'chu_de': 'Giao dịch điện tử', 'key_words': ['luật', 'quốc', 'hội', '20/2023/QH15', 'thương', 'mại', 'công', 'nghệ', 'thông', 'tin', 'quyền', 'dân', 'sự', '22/06/2023', '01/07/2024', 'giao', 'dịch', 'điện', 'tử', 'điều', '1', 'phạm', 'vi', 'chỉnh', 'này', 'quy', 'định', 'việc', 'thực', 'hiện', 'bằng', 'phương', 'tiện', 'chương', 'I', 'những', 'chung', 'khoản', '20-2023-QH15', '22-06-2023', '01-07-2024'], 'Chapter': 'Chương I: NHỮNG QUY ĐỊNH CHUNG', 'Section': None, 'Mini-Section': None, 'Article': 'Điều 1. Phạm vi điều chỉnh', 'Content': '1. Luật này quy định việc thực hiện giao dịch bằng phương tiện điện tử.', 'Article-Section': 'Khoản 1.'}
------

## 3.3. Tạo collection

In [9]:
qdrant = QdrantVectorStore.from_documents(
    documents = documents,
    embedding = model_Dense_Embedding,
    url = url,
    api_key = api_key,
    prefer_grpc=True,
    collection_name = "demo_Luat_Keywords",
    retrieval_mode=RetrievalMode.DENSE,
	metadata_payload_key="metadata"
)